Написать гибридную рекомендательную систему для датасета ml-latest

In [1]:
import numpy as np
import pandas as pd

from surprise import AlgoBase, KNNWithMeans, SVD, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
users = pd.read_csv(
    'users.dat',
    delimiter='::',
    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
    index_col=False
                   )
users

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [3]:
movies = pd.read_csv(
    'movies.dat',
    delimiter='::',
    names=['MovieID', 'Title', 'Genres'],
    index_col=False,
    encoding='latin-1'
                   )
movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [4]:
ratings = pd.read_csv(
    'ratings.dat',
    delimiter='::',
    names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
    index_col=False
                   )
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


Я попробую  получить взвешенную оценку по двум алгоритмам и посмотреть, удастся ли таким образом снизить rmse лучшего из использованных алгоритмов.
В качестве исходных алгоритмов я выбрала SVD и KNNWithMeans

In [5]:
dataset = pd.DataFrame({
    'uid': ratings.UserID,
    'iid': ratings.MovieID,
    'rating': ratings.Rating
})

In [6]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
data

In [7]:
trainset, testset = train_test_split(data, test_size=.2)

In [8]:
algo = SVD(n_factors=50, n_epochs=19, lr_all=0.0055, reg_all=0.03)
algo.fit(trainset)

In [9]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8668


0.866792238827765

In [10]:
algo.predict(uid=1, iid=3)

Prediction(uid=1, iid=3, r_ui=None, est=3.1375796989579956, details={'was_impossible': False})

In [11]:
algo_2 = KNNWithMeans(k=35, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo_2.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [12]:
test_pred_2 = algo_2.test(testset)
accuracy.rmse(test_pred_2, verbose=True)

RMSE: 0.8895


0.8895172095474834

In [13]:
algo_2.predict(uid=1, iid=2)

Prediction(uid=1, iid=2, r_ui=None, est=3.4893819768098213, details={'actual_k': 35, 'was_impossible': False})

In [14]:
# Я не смогу воспользоваться встроенной функцией для расчета RMSE,
# потому что придется перевести данные в изменяемый формат.
def rmse(dct):
    sum_mse = 0
    for pred in dct:
        mse = (pred['r_ui'] - pred['est']) ** 2
        sum_mse += mse

    rmse = np.sqrt(sum_mse / len(dct))
    return rmse

In [15]:
#Словарь, в котором будет подсчитываться взвешенная оценка
test_pred_combinations = [{'uid': x.uid, 'iid': x.iid, 'r_ui': x.r_ui, 'est': x.est} for x in test_pred]

#Цикл для подсчета RMSE при разных весах алгоритмов
for i in range(5, 10):
    for pred in test_pred_combinations:
        est_svd = algo.predict(uid=pred['uid'], iid=pred['iid']).est
        est_knn = algo_2.predict(uid=pred['uid'], iid=pred['iid']).est
        pred['est'] = (est_svd * i / 10) + (est_knn * (10 - i) / 10)
    print(f'weghts: est_svd={i / 10}, est_knn = {(10 - i) / 10}')
    print(rmse(test_pred_combinations))
    print()

weghts: est_svd=0.5, est_knn = 0.5
0.8657681742928534

weghts: est_svd=0.6, est_knn = 0.4
0.8639632719636922

weghts: est_svd=0.7, est_knn = 0.3
0.8631621878369519

weghts: est_svd=0.8, est_knn = 0.2
0.8633677161246057

weghts: est_svd=0.9, est_knn = 0.1
0.8645791389493508



Таким образом, при использовании оценок двух алгоритмов в сочетании SVD=0.7 и KNN=0.3, удалось снизить RMSE, полученное при использовании SVD, почти на 0.04. Проверим, отразилось ли это улучшение на топ-15 рекомендуемых пользователю фильмов.

In [16]:
#Чтобы иметь возможность обращаться к рекомендациям по всем парам пользователь-фильм, пришлось запустить этот огромный цикл.
#Надеюсь, существует более экономный способ получить взвешенные рекомендации нескольких алгоритмов
combine_pred = dict()
for user in users.UserID.unique():
    for movie in movies.MovieID.unique():
        estim = algo.predict(uid=user, iid=movie).est * 0.7 + algo_2.predict(uid=user, iid=movie).est * 0.3
        combine_pred[(user, movie)] = estim
combine_pred

{(1, 1): 4.588283637455241,
 (1, 2): 3.59486946190035,
 (1, 3): 3.3537302454068056,
 (1, 4): 3.245129621539509,
 (1, 5): 3.538838163696581,
 (1, 6): 4.243724058845245,
 (1, 7): 3.893374470358384,
 (1, 8): 3.395974806243098,
 (1, 9): 3.093686163303798,
 (1, 10): 3.8940523956139637,
 (1, 11): 4.219281639902164,
 (1, 12): 2.743968634759813,
 (1, 13): 3.7377693254728515,
 (1, 14): 3.8348160263195714,
 (1, 15): 3.0334577000229412,
 (1, 16): 4.089945003977171,
 (1, 17): 4.2938803756227735,
 (1, 18): 3.7338396448402316,
 (1, 19): 2.8492518947184484,
 (1, 20): 3.135805820141991,
 (1, 21): 3.880648900390854,
 (1, 22): 3.798265072306548,
 (1, 23): 3.3416697927685495,
 (1, 24): 3.714680870459035,
 (1, 25): 3.8491104348241736,
 (1, 26): 4.024403485343465,
 (1, 27): 3.500521804151803,
 (1, 28): 4.334849892983376,
 (1, 29): 4.158886151838813,
 (1, 30): 4.064567862372581,
 (1, 31): 3.4807311210917797,
 (1, 32): 4.1564026647296535,
 (1, 33): 3.7621735214807788,
 (1, 34): 4.148400989016556,
 (1, 35): 3

In [17]:
#Рекомендации комбинированного алгоритма
current_user_id = 19

user_movies = ratings[ratings.UserID == current_user_id].MovieID.unique()

estims = dict()

for movie in ratings.MovieID.unique():
    if movie in user_movies:
        continue
    
    else:
        title = movies.loc[movies.MovieID==movie, 'Title'].values[0]
        estims[title] = combine_pred[(19, movie)]

for est in sorted(estims, key=lambda x: estims[x], reverse=True)[:15]:
    print(f'{est}: {estims[est]}')

Sanjuro (1962): 4.692634183595266
For All Mankind (1989): 4.521715484432226
Usual Suspects, The (1995): 4.47518616877946
Monty Python and the Holy Grail (1974): 4.356385619693576
Hearts and Minds (1996): 4.354356798037244
Close Shave, A (1995): 4.349358251349303
Pulp Fiction (1994): 4.342174737279369
Green Mile, The (1999): 4.339942709446522
Apple, The (Sib) (1998): 4.326472540393821
Some Folks Call It a Sling Blade (1993): 4.31156822951625
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963): 4.302509839395123
Henry V (1989): 4.291469851557675
Palm Beach Story, The (1942): 4.287071723124484
Gladiator (2000): 4.2853947075762155
Killer, The (Die xue shuang xiong) (1989): 4.2763145521743695


In [18]:
#Рекомендации SVD для сравнения
current_user_id = 19
user_movies = ratings[ratings.UserID == current_user_id].MovieID.unique()

estims = dict()

for movie in ratings.MovieID.unique():
    if movie in user_movies:
        continue
    
    else:
        estims[movies.loc[movies.MovieID==movie, 'Title'].values[0]] = algo.predict(uid=current_user_id, iid=movie).est

for est in sorted(estims, key=lambda x: estims[x], reverse=True)[:15]:
    print(f'{est}: {estims[est]}')

Sanjuro (1962): 4.720944047259033
For All Mankind (1989): 4.486847790138426
Usual Suspects, The (1995): 4.462463149640945
Dersu Uzala (1974): 4.43341601166644
Gladiator (2000): 4.3951682857893015
Close Shave, A (1995): 4.3482671816352365
World of Apu, The (Apur Sansar) (1959): 4.347085979394536
Killer, The (Die xue shuang xiong) (1989): 4.342082015204345
Pather Panchali (1955): 4.338838208843447
Patton (1970): 4.335178800955979
Pulp Fiction (1994): 4.331553337656271
Monty Python and the Holy Grail (1974): 4.326629091273288
Blade Runner (1982): 4.319505016969747
Palm Beach Story, The (1942): 4.315397086791318
Green Mile, The (1999): 4.312383533857242


Таким образом, уменьшение погрешности на четыре сотых привело к довольно важным изменениям в списке рекомендаций для пользователя с id 19: изменился порядок фильмов, произошло несколько замен в рейтинге.